In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append("/Users/steeeve/Documents/GMU/GGS590/Final_Project")

os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import torch 
import ee
import pandas as pd
from MappingCropField import extract_metadata, generate_delineation, display_assigned_field

##### Config

In [ ]:
config = {"all_img_path": "/Users/steeeve/Documents/GMU/GGS590/Final_Project/data/all_sv_imgs",
          "color_composite": "true_color", # Pick between true_color and false_color
          "save_path": "/Users/steeeve/Documents/GMU/GGS590/Final_Project/sent2/",
          "credential_path": "/Users/steeeve/Documents/GMU/GGS590/Final_Project/hanxli_creds.json",
          "label_csv_path": "/Users/steeeve/Documents/GMU/GGS590/Final_Project/data/csiss_imgs.csv",
          "folder_id": "1iwSPV5D05rA9T2TdQ9gy9ED5UDimLpNA" # Folder ID from Google Drive Folder Address
          }

##### Extracting Metadata from Geotagged Images

In [ ]:
meta_df = extract_metadata(config["all_img_path"])
meta_df.head(3)

In [ ]:
label_df = pd.read_csv(config["label_csv_path"], index_col=0)
label_df.head(3)

In [ ]:
label_subset = label_df[["img_name", "crop_type"]]

# Merge df1 with the subset of df2 based on filename (df1) and img_name (df2)
meta_df = pd.merge(meta_df, label_subset, left_on="filename", right_on="img_name", how="left", suffixes=("_meta", "_label"))

# Drop the redundant img_name column
meta_df = meta_df.drop(columns=["img_name"])

# Display the updated DataFrame
meta_df = meta_df.dropna(subset=["crop_type"]).reset_index(drop=True)
meta_df.head(3)

##### Spatial processing

In [ ]:
ee.Authenticate()
ee.Initialize(project="ee-hanxli")

In [ ]:
generate_delineation(meta_df[0:1], 
                     color_composite = config["color_composite"], 
                     save_path = config["save_path"],
                     credentials_file_path = config["credential_path"],
                     folder_id = config["folder_id"],
                     visualize = True)

In [ ]:
lbl_bounds = display_assigned_field(meta_df, 
                                    color_composite = config["color_composite"], 
                                    save_path = config["save_path"],
                                    credentials_file_path = config["credential_path"],
                                    folder_id = config["folder_id"],
                                    batch_process=True)